# Financial Data Collection #

In [1]:
# lib imports
import pandas as pd
import yfinance as YF
from fredapi import Fred


In [2]:
# define key variables
fred_api = Fred("7c1f3f043ef85e198d4c6c17e0d8109e")
tickers = ['AAPL', 'PLTR', 'WMT', 'META', 'AMZN', 'LMT']

# func to obtain stock data info from yahoo finance
def get_tickers(tickers=[], period='1d'):
    # get tickers from yahoo finance
    table = pd.DataFrame()
    for ticker in tickers:
        data = YF.Ticker(ticker)
        df = data.history(period=period)
        df['ticker'] = ticker
        table = pd.concat([table, df])
    return table

In [4]:
# pull data into df and observe
stocks = get_tickers(tickers, period='max')
stocks.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,47952.000000,47952.000000,47952.000000,47952.000000,4.795200e+04,47952.000000,47952.000000
mean,50.388546,50.924328,49.848273,50.396994,1.028580e+08,0.003375,0.001494
std,95.971272,96.949773,94.967621,95.967293,2.106035e+08,0.075991,0.107540
min,0.000000,0.002831,0.002610,0.002831,0.000000e+00,0.000000,0.000000
25%,1.016131,1.033875,0.995973,1.016872,2.828725e+06,0.000000,0.000000
50%,10.886679,11.001080,10.762631,10.887998,2.608245e+07,0.000000,0.000000
75%,45.007633,45.406471,44.629456,45.005085,1.061082e+08,0.000000,0.000000
max,735.345771,740.251419,724.975032,736.015198,7.421641e+09,3.300000,20.000000


# Push into Database #